In [8]:
import pandas as pd
import numpy as np

# TODO: Alex to change this function into a class; utilize OOP methods and split this into helper functions to improve future debugging
# TODO: Alex to add unit tests for this process and validation processes (other than current 1 exception case)

def cleanQualtricsData(raw, roster, question_dictionary, needsMapping=True, needsInversion=True, needsNormalization=True):

    #assert all files are of .csv extension 
    #if not any(["CSV" in [raw.upper(), roster.upper(), question_dictionary.upper()]]):
    #    raise Exception("At least one of the input files is not in CSV format")
    

    # Imports and Instantiations
    raw = pd.read_csv(raw)
    question_dictionary = pd.read_csv(question_dictionary)
    roster =  pd.read_csv(roster)
    quantitative_questions = list(question_dictionary[question_dictionary["type"] == "quantitative"]["question_id"])
    
    # Subset raw data to just student email, student first name, student last name, and all question responses 
    cols_needed = []
    keywords_list = ['EMAIL', 'FIRST', 'LAST', 'Q']
    for col in list(raw.columns):
        if any(keyword in col.upper() for keyword in keywords_list):
            cols_needed.append(col)
    subset_data = raw[cols_needed]

    # Replace question column names in subset data with X.Y instead of QX_Y
    subset_data.columns = [col.replace('Q', '').replace('_', '.') for col in list(subset_data.columns)]

    # Instantiate cleaned, a pointer of subset_data
    cleaned = subset_data

    # Join in TeamNumber and TeammateNumber from roster; drop rows of metadata
    roster_email_field = [col for col in list(roster.columns) if 'EMAIL' in col.upper()][0]
    cleaned_email_field = [col for col in list(cleaned.columns) if 'EMAIL' in col.upper()][0]
    

    full_cleaned = pd.merge(roster[[roster_email_field, 'TeamNumber', 'TeammateNumber']], 
                            cleaned, 
                            how="outer", 
                            left_on = roster_email_field, 
                            right_on = cleaned_email_field)
    full_cleaned = full_cleaned[~full_cleaned["TeamNumber"].isna()]

    # Sort df by TeamNumber then TeammateNumber starting from Team1
    full_cleaned = full_cleaned.sort_values(["TeamNumber", "TeammateNumber"]).reset_index().drop("index", axis=1)
    
    # If the raw data has "Agree"/"Disagree" in quantitative question columns, map these to integers 1-X where X is "out_of"
    if needsMapping:
        seven_scale_mappings = {
            "STRONGLY AGREE" : 7,
            "AGREE" : 6,
            "SOMEWHAT AGREE" : 5,
            "NEITHER AGREE NOR DISAGREE" : 4,
            "SOMEWHAT DISAGREE" : 3,
            "DISAGREE" : 2,
            "STRONGLY DISAGREE" : 1,
            np.nan : np.nan,
            "MUCH BETTER" : 7,
            "MODERATELY BETTER" : 6,
            "SLIGHTLY BETTER" : 5,
            "ABOUT THE SAME" : 4,
            "SLIGHTLY WORSE" : 3,
            "MODERATELY WORSE" : 2,
            "MUCH WORSE" : 1,
        }
        five_scale_mappings = {
            "ALWAYS" : 5,
            "VERY OFTEN" : 4,
            "SOMETIMES" : 3,
            "RARELY" : 2,
            "NEVER" : 1,
            np.nan : np.nan
        }

        for question in quantitative_questions:
            denominator = int(question_dictionary[question_dictionary["question_id"] == question]["out_of"])
            question_str = str(question)
            
            if question_str not in ('38.1', '38.2', '38.3'):
                if denominator == 7:
                    full_cleaned[question_str] = full_cleaned[question_str].str.upper()
                    full_cleaned[question_str] = [seven_scale_mappings[response] for response in full_cleaned[question_str]]
                if denominator == 5:
                    full_cleaned[question_str] = full_cleaned[question_str].str.upper()
                    full_cleaned[question_str] = [five_scale_mappings[response] for response in full_cleaned[question_str]]

                # if question is quantitative but dtype is a str, change data type
                if full_cleaned[[question_str]].dtypes[0] == str:
                    full_cleaned[question_str] = pd.to_numeric(full_cleaned[question_str])
        
   # If needs inversion is true, then for all 5 point and 7 point scales, change 1s to 7s/5s, 2s to 6s/4s, etc. 
   # Due to the way percentiles are calculated, a 7 should ALWAYS indicate STRONGLY AGREE
    if needsInversion:
        for question in quantitative_questions:
            denominator = int(question_dictionary[question_dictionary["question_id"] == question]["out_of"])
            question_str = str(question)
            
            if denominator == 7:
                full_cleaned[question_str] = (8 - pd.to_numeric(full_cleaned[question_str]))
            if denominator == 5:
                full_cleaned[question_str] = (6 - pd.to_numeric(full_cleaned[question_str]))
    
    # If normalization is true, then all quantitative data is normed to 0. Assumed question_dictionary includes "out_of" column
    if needsNormalization:
        for question in quantitative_questions:
            denominator = int(question_dictionary[question_dictionary["question_id"] == question]["out_of"])
            question_str = str(question)
            
            if question not in (38.1, 38.2, 38.3):
                if denominator == 7:
                    full_cleaned[question_str] = (pd.to_numeric(full_cleaned[question_str]) - 4) * -1
                if denominator == 5:
                    full_cleaned[question_str] = (pd.to_numeric(full_cleaned[question_str]) - 3) * -1
                
    # For NA values (students that didn't complete survey, left question empty), fill in with "No Response"
    full_cleaned = full_cleaned.fillna('No Response')
    
    return full_cleaned.to_csv("full_cleaned.csv")



In [9]:
data = pd.read_csv("ME103_CHECKIN01_RAW.csv")
data.columns

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q2_1', 'Q2_2', 'Q2_3', 'Q3',
       'Q3_5', 'Q4_1', 'Q4_5', 'Q4_6', 'Q4_2', 'Q4_3', 'Q4_4', 'Q4_7', 'Q5_1',
       'Q5_2', 'Q5_3', 'Q5_4', 'Q5_5', 'Q5_6', 'Q6', 'Q7', 'Q14_1', 'Q14_2',
       'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q15_1', 'Q15_2', 'Q15_3', 'Q15_4',
       'Q15_5', 'Q15_6', 'Q16_1', 'Q16_2', 'Q16_3', 'Q16_4', 'Q16_5', 'Q16_6',
       'Q17_1', 'Q17_2', 'Q17_3', 'Q17_4', 'Q17_5', 'Q17_6', 'Q18_1', 'Q18_2',
       'Q18_3', 'Q18_4', 'Q18_5', 'Q18_6', 'Q19_1', 'Q19_2', 'Q19_3', 'Q19_4',
       'Q19_5', 'Q19_6', 'Q25'],
      dtype='object')

In [10]:
cleanQualtricsData("ME103_CHECKIN01_RAW.csv", 
                   'ME103_ROSTER.CSV',
                   'ME103_QUESTION_DICTIONARY.csv', 
                   needsMapping=False,
                   needsInversion=True,
                   needsNormalization=False)